# Demand and Inventory Control 


### The problem <br>

A Supply Chain department for a retail company needs to balance their inventory and demand such that <br>
there is also enough inventory to match the demand but the inventory is not allowed to go below a certain level <br>
nor above a certain level.


In [ ]:
import os

import pandas as pd
import polars as pl
import itertools
import numpy as np


from src.supplyseer.optimization.cpsat import SupplyChainModel, SupplyChainSolver

### Let's generate some fake data

In [ ]:
# Example usage with Pandas or Polars DataFrame
# Create combinations of product and location IDs
product_ids = [f'P{i}' for i in range(1, 151)]
location_ids = [f'L{i}' for i in range(1, 101)]
combinations = list(itertools.product(product_ids, location_ids))


In [6]:
# This is the product-location mapping
combinations[:5]

[('P1', 'L1'), ('P1', 'L2'), ('P1', 'L3'), ('P1', 'L4'), ('P1', 'L5')]

In [32]:
df = pd.DataFrame({
    'product_id': [combo[0] for combo in combinations],
    'location_id': [combo[1] for combo in combinations],
    'initial_stock': np.random.poisson(200),
    'min_stock': np.random.poisson(50),
    'max_stock': np.random.poisson(1000),
    'holding_cost': 2,
    'max_production': 500,
    'production_cost': 10
})


In [34]:
df.head()

,product_id,location_id,initial_stock,min_stock,max_stock,holding_cost,max_production,production_cost
0,P1,L1,206,58,990,2,500,10
1,P1,L2,206,58,990,2,500,10
2,P1,L3,206,58,990,2,500,10
3,P1,L4,206,58,990,2,500,10
4,P1,L5,206,58,990,2,500,10


In [35]:
# Assume we have another DataFrame for demand over time
# Each row represents a demand value for a specific product, location, and time period
# Columns: 'product_id', 'location_id', 'time_period', 'demand'
demand_df = pd.DataFrame({
    'product_id': [combo[0] for combo in combinations for _ in range(7)],
    'location_id': [combo[1] for combo in combinations for _ in range(7)],
    'time_period': list(range(7)) * len(combinations),
    'demand': [100 + (i * j) % 300 for i, j in itertools.product(range(1, 151), range(1, 101)) for _ in range(7)]
})


In [36]:
demand_df

,product_id,location_id,time_period,demand
0,P1,L1,0,101
1,P1,L1,1,101
2,P1,L1,2,101
3,P1,L1,3,101
4,P1,L1,4,101
...,...,...,...,...
104995,P150,L100,2,100
104996,P150,L100,3,100
104997,P150,L100,4,100
104998,P150,L100,5,100


In [37]:
products = df['product_id'].unique().tolist()
locations = df['location_id'].unique().tolist()

In [38]:
total_inventory_cost = 0
total_production_cost = 0

In [39]:
# Group demand by product and location for efficient lookup
demand_grouped = demand_df.groupby(['product_id', 'location_id'])

### Initialize the Supply Chain model with products and locations

In [40]:
model = SupplyChainModel(products, locations)

In [41]:
# Add constraints for each product-location pair based on DataFrame
for product_id, location_id, min_stock, max_stock, initial_stock, holding_cost, max_production, production_cost in zip(
        df['product_id'], df['location_id'], df['min_stock'], df['max_stock'], df['initial_stock'], df['holding_cost'],
        df['max_production'], df['production_cost']):
    
    inv_cost = model.add_inventory_constraint(product_id, location_id, min_stock, max_stock, initial_stock, holding_cost)
    prod_cost = model.add_production_constraint(product_id, location_id, max_production, production_cost)
    
    # Retrieve demand series without pivoting
    demand_series = demand_grouped.get_group((product_id, location_id)).sort_values(by='time_period')['demand']
    model.add_demand_constraint(product_id, location_id, demand_series)
    
    total_inventory_cost += inv_cost
    total_production_cost += prod_cost

### Set the objective function and initialize the solver

- Minimize the total inventory cost (cost of holding inventory)
- Minimize the total production cost (cost of producing goods or buying goods)

In [42]:
# Set the objective
model.set_objective(total_inventory_cost, total_production_cost)

In [43]:
# Solve the model
solver = SupplyChainSolver()

In [44]:
status = solver.solve_with_timeout(model, 60)  # 60 seconds timeout

In [48]:
if status == 4 or status == 2:
    solver.print_solution(model)
else:
    print("No solution found.")

print("\nSolution Summary:", solver.get_solution_summary())

Solution:
Product P1 at Location L1:
  Inventory levels: [206, 105, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 54, 101, 101, 101, 101]
Product P1 at Location L2:
  Inventory levels: [206, 104, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 56, 102, 102, 102, 102]
Product P1 at Location L3:
  Inventory levels: [206, 103, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 58, 103, 103, 103, 103]
Product P1 at Location L4:
  Inventory levels: [206, 102, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 60, 104, 104, 104, 104]
Product P1 at Location L5:
  Inventory levels: [206, 101, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 62, 105, 105, 105, 105]
Product P1 at Location L6:
  Inventory levels: [206, 100, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 64, 106, 106, 106, 106]
Product P1 at Location L7:
  Inventory levels: [206, 99, 58, 58, 58, 58, 58]
  Production quantities: [0, 0, 66, 107, 107, 107, 107]
Product P1 at Location L8:
  Inventory levels: [206, 

In [60]:
totalcost = solver.get_solution_summary()["best_objective"]
avgprod = totalcost / len(products) # Average cost per product
avgloc = totalcost / len(locations) # Average cost per location
avgpair = totalcost / (len(products) * len(locations)) # Average cost per product-location pair

print(f"Avg cost per product: {avgprod:.2f}")
print(f"Avg cost per location: {avgloc:.2f}")
print(f"Avg cost per product-location pair: {avgpair:.2f}")

Avg cost per product: 1430647.83
Avg cost per location: 2145971.74
Avg cost per product-location pair: 14306.48


### Conclusion

- The total cost optimized for lands at 214 597 174.
- It is the optimal solution which means it hasn't found a lower total cost than that solution

* Average total cost per product: 1 430 647
* Average total cost per location: 2 145 971
* Average total cost per product-location pair: 14 306